

*   **Please first download the CSV file from Github repo,**
*   **Then upload the file in the G-drive and mount them as well**



In [7]:
df_AAC= pd.read_csv("/content/drive/MyDrive/kan/AAC_all.csv")
df_AESNN= pd.read_csv("/content/drive/MyDrive/kan/AESNN3_all.csv")
df_Zscale= pd.read_csv("/content/drive/MyDrive/kan/Zscale_all.csv")
df_Blosume= pd.read_csv("/content/drive/MyDrive/kan/blosume62_all.csv")
df_OPF= pd.read_csv("/content/drive/MyDrive/kan/opf10bit_all.csv")
df_merged= pd.read_csv("/content/drive/MyDrive/kan/merged all.csv")
df_neuroAAC= pd.read_csv("/content/drive/MyDrive/Antifungal code/Neuropeptide/aactrain.csv")

**KAN [ kormologov analond model ]**
installation procedure from pykan doccumentation, for more details :  https://kindxiaoming.github.io/pykan/Examples/Example_3_classfication.html

**Requirment:**


*   torch==2.2.1+cu121


---


*  matplotlib==3.7.1


---



*   sklearn==1.2.2


---



*   moviepy==1.0.3




In [2]:
!pip install pykan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 2.1 MB/s eta 0:00:00


**Check CUDA and CPU**
```
1.   if torch.cuda.is_available():
  device = torch.device("cuda")
2.   else:
  device = torch.device("cpu")

print(device)

```





In [3]:
import torch
from kan import *
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import moviepy.video.io.ImageSequenceClip

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

cpu


In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.preprocessing import StandardScaler

Convert to PyTorch tensors
---
Concatenate all data into a single tensor on the specified device
---








In [8]:

import csv
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

def load_iris_dataset(csv_file):

    iris = pd.read_csv(csv_file)
    data_tensor = torch.tensor(iris.iloc[:, :-1].values, dtype=torch.float)
    target_tensor = torch.tensor(iris.iloc[:, -1].values, dtype=torch.long)

    # Convert to PyTorch tensors
    #data_tensor = torch.tensor(data, dtype=torch.float32)
    #target_tensor = torch.tensor(target, dtype=torch.long)

    # Split dataset into train and test sets
    train_data= train, test_data, train_target, test_target = train_test_split(data_tensor, target_tensor, test_size=0.2, random_state=42)

    # Create data loaders (optional, if you want to batch and shuffle the data)
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data, train_target), batch_size=1, shuffle=True)
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data, test_target), batch_size=1, shuffle=False)

    train_inputs = torch.empty(0, 1168, device=device)
    train_labels = torch.empty(0, dtype=torch.long, device=device)
    test_inputs = torch.empty(0, 1168, device=device)
    test_labels = torch.empty(0, dtype=torch.long, device=device)

    # Concatenate all data into a single tensor on the specified device
    for data, labels in train_loader:
        train_inputs = torch.cat((train_inputs, data.to(device)), dim=0)
        train_labels = torch.cat((train_labels, labels.to(device)), dim=0)

    for data, labels in test_loader:
        test_inputs = torch.cat((test_inputs, data.to(device)), dim=0)
        test_labels = torch.cat((test_labels, labels.to(device)), dim=0)

    dataset = {}
    dataset['train_input'] = train_inputs
    dataset['test_input'] = test_inputs
    dataset['train_label'] = train_labels
    dataset['test_label'] = test_labels

    return dataset

iris_dataset = df_Zscale

In [10]:
import csv
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import KFold

def load_iris_dataset(csv_file):
    # Load the dataset
    iris = df_Zscale
    data_tensor = torch.tensor(iris.iloc[:, :-1].values, dtype=torch.float)
    target_tensor = torch.tensor(iris.iloc[:, -1].values, dtype=torch.long)

    # Use KFold for cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = {
        'folds': []
    }

    # Iterate through each fold
    for fold, (train_indices, test_indices) in enumerate(kfold.split(data_tensor)):
        # Split data
        train_data, test_data = data_tensor[train_indices], data_tensor[test_indices]
        train_target, test_target = target_tensor[train_indices], target_tensor[test_indices]

        # Create data loaders
        train_loader = DataLoader(TensorDataset(train_data, train_target), batch_size=10, shuffle=True)
        test_loader = DataLoader(TensorDataset(test_data, test_target), batch_size=10, shuffle=True)

        train_inputs = torch.empty(0, train_data.size(1), device=device)
        train_labels = torch.empty(0, dtype=torch.long, device=device)
        test_inputs = torch.empty(0, test_data.size(1), device=device)
        test_labels = torch.empty(0, dtype=torch.long, device=device)

        # Concatenate all data into a single tensor on the specified device
        for data, labels in train_loader:
            train_inputs = torch.cat((train_inputs, data.to(device)), dim=0)
            train_labels = torch.cat((train_labels, labels.to(device)), dim=0)

        for data, labels in test_loader:
            test_inputs = torch.cat((test_inputs, data.to(device)), dim=0)
            test_labels = torch.cat((test_labels, labels.to(device)), dim=0)

        # Append the fold data to the dataset
        dataset['folds'].append({
            'train_input': train_inputs,
            'test_input': test_inputs,
            'train_label': train_labels,
            'test_label': test_labels
        })

    return dataset

# Correctly calling the function
csv_file = df_Zscale
iris_dataset = load_iris_dataset(csv_file)

# Printing the shapes of the data tensors for each fold
for fold_idx, fold_data in enumerate(iris_dataset['folds']):
    print(f"Fold {fold_idx + 1}")
    print("Train data shape: {}".format(fold_data['train_input'].shape))
    print("Train target shape: {}".format(fold_data['train_label'].shape))
    print("Test data shape: {}".format(fold_data['test_input'].shape))
    print("Test target shape: {}".format(fold_data['test_label'].shape))


Fold 1
Train data shape: torch.Size([362, 155])
Train target shape: torch.Size([362])
Test data shape: torch.Size([41, 155])
Test target shape: torch.Size([41])
Fold 2
Train data shape: torch.Size([362, 155])
Train target shape: torch.Size([362])
Test data shape: torch.Size([41, 155])
Test target shape: torch.Size([41])
Fold 3
Train data shape: torch.Size([362, 155])
Train target shape: torch.Size([362])
Test data shape: torch.Size([41, 155])
Test target shape: torch.Size([41])
Fold 4
Train data shape: torch.Size([363, 155])
Train target shape: torch.Size([363])
Test data shape: torch.Size([40, 155])
Test target shape: torch.Size([40])
Fold 5
Train data shape: torch.Size([363, 155])
Train target shape: torch.Size([363])
Test data shape: torch.Size([40, 155])
Test target shape: torch.Size([40])
Fold 6
Train data shape: torch.Size([363, 155])
Train target shape: torch.Size([363])
Test data shape: torch.Size([40, 155])
Test target shape: torch.Size([40])
Fold 7
Train data shape: torch.Siz

# Model applied of each feature extraction method
we used here 10 fold cross validation methods:

▶ where `model = KAN(width=[155, 5, 2], device='cuda' if torch.cuda.is_available() else 'cpu')`
here  in width [155= features number, 5= hidden layers,and  2= output layers, we used 2 cause we have a 0 and 1 binary classification dataset]

▶ most importantly here all the feature extraction methods have used **5= hidden layers,and  2= output layers** and feature number used based on the *quantity of the features*


▶ Train the model  with `for epoch in range(400)` see the code and comments also

▶ Calculate **mean and standard deviation** of metrics

In [11]:
class KAN(torch.nn.Module):
    def __init__(self, width, device):
        super(KAN, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(width[0], width[1]),
            torch.nn.ReLU(),
            torch.nn.Linear(width[1], width[2])
        )
        self.device = device
        self.to(device)

    def forward(self, x):
        return self.layers(x)

In [13]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.preprocessing import StandardScaler



model = KAN(width=[155, 5, 2], device='cuda' if torch.cuda.is_available() else 'cpu')




def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, dataset, device):
    def to_cpu(tensor):
        return tensor.detach().cpu().numpy()

    def accuracy(outputs, labels):
        preds = torch.argmax(outputs, dim=1)
        return (preds == labels).float().mean().item()

    def sensitivity(outputs, labels):
        preds = torch.argmax(outputs, dim=1)
        tn, fp, fn, tp = confusion_matrix(labels.cpu(), preds.cpu()).ravel()
        return tp / (tp + fn)

    def specificity(outputs, labels):
        preds = torch.argmax(outputs, dim=1)
        tn, fp, fn, tp = confusion_matrix(labels.cpu(), preds.cpu()).ravel()
        return tn / (tn + fp)

    def auc_score(outputs, labels):
        preds = torch.nn.functional.softmax(outputs, dim=1)[:, 1]
        return roc_auc_score(labels.cpu(), preds.cpu())

    def mcc_score(outputs, labels):
        preds = torch.argmax(outputs, dim=1)
        return matthews_corrcoef(labels.cpu(), preds.cpu())

    def kappa_score(outputs, labels):
        preds = torch.argmax(outputs, dim=1)
        return cohen_kappa_score(labels.cpu(), preds.cpu())

    def precision(outputs, labels):
        preds = torch.argmax(outputs, dim=1)
        return precision_score(labels.cpu(), preds.cpu(), average='macro')

    def recall(outputs, labels):
        preds = torch.argmax(outputs, dim=1)
        return recall_score(labels.cpu(), preds.cpu(), average='macro')

    def f1(outputs, labels):
        preds = torch.argmax(outputs, dim=1)
        return f1_score(labels.cpu(), preds.cpu(), average='macro')

    metrics = {
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'auc': auc_score,
        'mcc': mcc_score,
        'kappa': kappa_score,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

    results = {f'train_{k}_list': [] for k in metrics.keys()}
    results.update({f'test_{k}_list': [] for k in metrics.keys()})

    for fold_data in dataset['folds']:
        train_inputs, train_labels = fold_data['train_input'], fold_data['train_label']
        test_inputs, test_labels = fold_data['test_input'], fold_data['test_label']

        # Prepare data loaders
        train_dataset = TensorDataset(train_inputs, train_labels)
        test_dataset = TensorDataset(test_inputs, test_labels)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

        # Define criterion and optimizer
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        # Train the model
        for epoch in range(40):  # with 400 epoch
            train_model(model, train_loader, criterion, optimizer, device)

        # Evaluate on training and testing data
        model.eval()
        with torch.no_grad():
            train_outputs = model(train_inputs.to(device))
            test_outputs = model(test_inputs.to(device))

        for metric_name, metric_func in metrics.items():
            train_metric_value = metric_func(train_outputs, train_labels.to(device))
            test_metric_value = metric_func(test_outputs, test_labels.to(device))
            results[f'train_{metric_name}_list'].append(train_metric_value)
            results[f'test_{metric_name}_list'].append(test_metric_value)

    # Calculate mean and standard deviation of metrics
    summary_results = {}
    for metric_name in metrics.keys():
        summary_results[f'train_{metric_name}_mean'] = np.mean(results[f'train_{metric_name}_list'])
        summary_results[f'train_{metric_name}_std'] = np.std(results[f'train_{metric_name}_list'])
        summary_results[f'test_{metric_name}_mean'] = np.mean(results[f'test_{metric_name}_list'])
        summary_results[f'test_{metric_name}_std'] = np.std(results[f'test_{metric_name}_list'])

    return summary_results

def load_iris_dataset(df_Zscale):

    # Convert the DataFrame to TensorDataset
    X = df_Zscale.iloc[:, :-1].values
    y = df_Zscale.iloc[:, -1].values
    X = StandardScaler().fit_transform(X)
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    # Create K-Folds
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    folds = []
    for train_idx, test_idx in kfold.split(X):
        folds.append({
            'train_input': X[train_idx],
            'train_label': y[train_idx],
            'test_input': X[test_idx],
            'test_label': y[test_idx]
        })
    return {'folds': folds}

iris_dataset = load_iris_dataset(df_Zscale)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

results = evaluate_model(model, iris_dataset, device)
print("Train Accuracy: {:.4f} ± {:.4f}".format(results['train_accuracy_mean'], results['train_accuracy_std']))
print("Test Accuracy: {:.4f} ± {:.4f}".format(results['test_accuracy_mean'], results['test_accuracy_std']))
print("Train Sensitivity: {:.4f} ± {:.4f}".format(results['train_sensitivity_mean'], results['train_sensitivity_std']))
print("Test Sensitivity: {:.4f} ± {:.4f}".format(results['test_sensitivity_mean'], results['test_sensitivity_std']))
print("Train Specificity: {:.4f} ± {:.4f}".format(results['train_specificity_mean'], results['train_specificity_std']))
print("Test Specificity: {:.4f} ± {:.4f}".format(results['test_specificity_mean'], results['test_specificity_std']))
print("Train AUC: {:.4f} ± {:.4f}".format(results['train_auc_mean'], results['train_auc_std']))
print("Test AUC: {:.4f} ± {:.4f}".format(results['test_auc_mean'], results['test_auc_std']))
print("Train MCC: {:.4f} ± {:.4f}".format(results['train_mcc_mean'], results['train_mcc_std']))
print("Test MCC: {:.4f} ± {:.4f}".format(results['test_mcc_mean'], results['test_mcc_std']))
print("Train Kappa: {:.4f} ± {:.4f}".format(results['train_kappa_mean'], results['train_kappa_std']))
print("Test Kappa: {:.4f} ± {:.4f}".format(results['test_kappa_mean'], results['test_kappa_std']))
print("Train Precision: {:.4f} ± {:.4f}".format(results['train_precision_mean'], results['train_precision_std']))
print("Test Precision: {:.4f} ± {:.4f}".format(results['test_precision_mean'], results['test_precision_std']))
print("Train Recall: {:.4f} ± {:.4f}".format(results['train_recall_mean'], results['train_recall_std']))
print("Test Recall: {:.4f} ± {:.4f}".format(results['test_recall_mean'], results['test_recall_std']))
print("Train F1 Score: {:.4f} ± {:.4f}".format(results['train_f1_mean'], results['train_f1_std']))
print("Test F1 Score: {:.4f} ± {:.4f}".format(results['test_f1_mean'], results['test_f1_std']))


Train Accuracy: 0.9760 ± 0.0079
Test Accuracy: 0.9145 ± 0.1367
Train Sensitivity: 0.9419 ± 0.0073
Test Sensitivity: 0.8763 ± 0.1862
Train Specificity: 0.9969 ± 0.0093
Test Specificity: 0.9457 ± 0.1066
Train AUC: 0.9911 ± 0.0043
Test AUC: 0.9393 ± 0.1070
Train MCC: 0.9495 ± 0.0171
Test MCC: 0.8280 ± 0.2774
Train Kappa: 0.9486 ± 0.0169
Test Kappa: 0.8238 ± 0.2813
Train Precision: 0.9802 ± 0.0095
Test Precision: 0.9172 ± 0.1369
Train Recall: 0.9694 ± 0.0076
Test Recall: 0.9110 ± 0.1406
Train F1 Score: 0.9743 ± 0.0084
Test F1 Score: 0.9110 ± 0.1417
